# MoBioFP - Fingertip Image Quality Assessment

In [ ]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from mobiofp.iqa import gradient_magnitude
from pathlib import Path

%matplotlib inline

In [ ]:
IMAGES_DIR = "IMAGES_DIR"
MASKS_DIR = "MASKS_DIR"
IQA_FILE = "IQA_FILE"
BRISQUE_MODEL = "../models/brisque_model_live.yml"
BRISQUE_RANGE = "../models/brisque_range_live.yml"

In [ ]:
def read_images(src_dir, image_names):
    images = []
    images_name = []
    for image_name in image_names:
        image = cv2.imread(f"{src_dir}/{image_name}", cv2.IMREAD_GRAYSCALE)
        images.append(image)
        images_name.append(image_name)

    return images, images_name


def plot_images(images, titles=None, rows=1, cols=None, sup_title=None, show_axis=False):
    num_images = len(images)

    if titles is None:
        titles = ["Image {}".format(i + 1) for i in range(num_images)]

    if cols is None:
        cols = num_images // rows + (1 if num_images % rows else 0)

    _, axes = plt.subplots(rows, cols, figsize=(cols * 4, rows * 4))

    for i, ax in enumerate(axes.flat):
        cmap = "gray" if len(images[i].shape) == 2 else None
        if i < num_images:
            ax.imshow(images[i], cmap=cmap)
            ax.set_title(titles[i])
            ax.axis("on" if show_axis else "off")
        else:
            # Do not show the remaining subplots
            ax.axis("off")

    if sup_title:
        plt.suptitle(sup_title)

    plt.tight_layout()
    plt.show()


def plot_dataframe(df, plot_func, title, suptitle, **kwargs):
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    for col, ax in zip(df.columns[1:], axes.flat):
        plot_func(df[col], ax=ax, **kwargs)
        ax.set_title(f"{title} of {col}")

    fig.suptitle(f"{title} of all columns in the dataframe")
    plt.tight_layout()
    plt.show()


def plot_correlation_heatmap(df):
    plt.figure(figsize=(10, 7))
    corr = df.drop(columns=["Image name"]).corr()
    mask = np.triu(np.ones_like(corr))
    heat = sns.heatmap(corr, annot=True, mask=mask, vmin=-1, vmax=1, cmap="BrBG")
    heat.set_title("Correlation Heatmap", fontdict={"fontsize": 12}, pad=12)
    plt.show()


def remove_outliers_iqr(df):
    for col in df.columns[1:]:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1

        # Filter out the outliers
        return df[(df[col] >= Q1 - 1.5 * IQR) & (df[col] <= Q3 + 1.5 * IQR)]

In [ ]:
df = pd.read_csv(IQA_FILE)
df.head()

In [ ]:
df.describe()

In [ ]:
plot_dataframe(df, sns.histplot, "Histogram", "Distribution", kde=True)
plot_dataframe(df, sns.boxplot, "Boxplot", "Distribution")
plot_correlation_heatmap(df)

In [ ]:
# Remove outliers using IQR
df_no_outliers = remove_outliers_iqr(df)
plot_dataframe(df_no_outliers, sns.histplot, "Histogram", "Distribution", kde=True)
plot_dataframe(df_no_outliers, sns.boxplot, "Boxplot", "Distribution", showfliers=False)
plot_correlation_heatmap(df_no_outliers)
df_no_outliers.describe()

In [ ]:
df2 = pd.DataFrame(
    columns=[
        "Image name",
        "Laplacian",
        "Noise",
        "Contrast",
        "BRISQUE",
        "Subject ID",
        "Illumination",
        "Finger ID",
        "Background",
        "Impression ID",
    ]
)

for i, row in df_no_outliers.iterrows():
    image_name = Path(row["Image name"]).stem
    subject_id, illumination, finger_id, background, impression_id = image_name.split("_")
    df2.loc[i] = [
        row["Image name"],
        row["Laplacian"],
        row["Noise"],
        row["Contrast"],
        row["BRISQUE"],
        subject_id,
        illumination,
        finger_id,
        background,
        impression_id,
    ]

In [ ]:
print(f"# of gallery images: {len(df2[df2['Background'] == 'w'])}")
print(f"# of probe images: {len(df2[df2['Background'] == 'n'])}")

In [ ]:
images, image_titles = read_images(IMAGES_DIR, df2["Image name"])
plot_images(images, image_titles, rows=2)

In [ ]:
gradient_magnitude_images = [gradient_magnitude(image) for image in images]
plot_images(gradient_magnitude_images, image_titles, rows=2)